In [3]:
import pandas as pd
from pylab import *
from collections import Counter
import scipy
from matplotlib import rc
import numpy as np
from sklearn import preprocessing
import pybedtools as pyb
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})

Loading STR data

In [4]:
Repeats = pd.read_table('../data/STRs_depth5_chr_start_maf09.annotated.tsv')

# Adding gene info to promoter rows
promoter = Repeats[Repeats['WHERE']=='PROMOTER']
rest = Repeats[Repeats['WHERE']!='Promoter']
promoter['GENE2'] = [x for x in promoter.Closest]
Repeats = pd.concat([promoter, rest]).reset_index()

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Making dataframes for matrix eQTL

In [ ]:
# "SNP" location data
STR_loc_data = Repeats[['CHR_START', 'CHR', 'START']].drop_duplicates().set_index('CHR_START')
STR_loc_data.to_csv('../data/STR_loc_data.tsv', sep = '\t')

# Gene expression data
Data = pd.read_table('../data/GSE80744_ath1001_tx_norm_2016-04-21-UQ_gNorm_normCounts_k4.tsv')
Data.columns = ['GENE2'] + [i.replace('X', '') for i in Data.columns if 'X' in i]

# Mapper 
mapper = Repeats.set_index('CHR_START')['GENE2'].to_dict()

# Mapping accession to ecotype
ACC_ECO = pd.read_csv('../data/AccListSraRunTable.csv')[['ACCESSION', 'Ecotype']].rename(columns = {'Ecotype' : 'ECOTYPE'})

#Mapping ecotype to accession ID
dicto = {}
for i in ACC_ECO.itertuples():
    dicto.update({i.ECOTYPE : i.ACCESSION})

# New colums for gene expression frame
new_columns = []
for i in Data.columns:
    if i == 'GENE2':
        new_columns.append(i)
    elif int(i) in dicto.keys():
        new_columns.append(dicto[int(i)])
    else:
        new_columns.append(i)
Data.columns = new_columns

# Removing accessions without STR data
Data.drop([col for col in Data.columns[1:] if 'SRR' not in col], axis=1,inplace=True)

# SNP data
STR_data_sub = Repeats[['CHR_START', 'ACCESSION', 'STR_DOSAGE']]

# Function to normalize STR data
def normalize(x):  
    missing_indices = [n for n, v in enumerate(x) if str(v) == 'nan']
    indices = [n for n, v in enumerate(x) if str(v) != 'nan']
    values = [v for n, v in enumerate(x) if str(v) != 'nan']
    scaled_values = preprocessing.scale(values)
    
    a = zeros(len(x))
    c = -1
    for n, v in enumerate(a):
        if n in missing_indices:
            a[n] = 'NaN'
        if n in indices:
            c += 1
            a[n] = scaled_values[c]

    return a

# Removing accessions without gene expression data
STR_data = STR_data_sub[STR_data_sub['ACCESSION'].isin(Data.columns[1:])].pivot_table(
           index = 'CHR_START', columns = 'ACCESSION', values = 'STR_DOSAGE')

# Normalizing
Norm_STR_data = STR_data.apply(normalize, axis = 1)

In [ ]:
# Not in Covariates
Norm_STR_data.drop(['SRR1945934'], axis = 1, inplace = True)
Norm_STR_data.to_csv('../data/Norm_STR_data.tsv', sep = '\t')

Control

In [ ]:
Control_data = Norm_STR_data.copy()
for n in range(len(Control_data.SRR1945451)):
    Control_data.iloc[n] = choice(Control_data.iloc[n], len(Control_data.iloc[n]), replace = False)

In [ ]:
# Not in Covariates
Control_data.to_csv('../data/Control_data.tsv', sep = '\t')

Number of samples need to be identical in STR_data file and GE_data file

In [ ]:
Data.drop([col for col in Data.columns[1:] if col not in STR_data.columns], axis=1,inplace=True)

In [ ]:
Data.drop(['SRR1945934'], axis = 1, inplace = True)
Data.set_index('GENE2').to_csv('../data/GE_data.tsv', sep = '\t')

Gene location data

In [ ]:
Gene_loc_data = Repeats[Repeats['GENE2'].isin(Data.GENE2)][['GENE2', 'CHR', 'START', 'END']].drop_duplicates()

In [ ]:
GFF = pyb.BedTool('../data/TAIR10_GFF3_genes.gff')
fh = open('../data/Gene_loc_data.tsv', 'w')
fh.write('GENE2' + '\t' + 'CHR' + '\t' + 'START' + '\t' + 'END' + '\n')
for i in GFF:
    if i[2] == 'gene':
        gene = i[-1].split('=')[1].split(';')[0]
        fh.write(gene + '\t' + i[0].lower() + '\t' + i[3] + '\t' + i[4] + '\n')
fh.close()
Gene_loc_data.to_csv('../data/Gene_loc_data.tsv', sep = '\t')

Population structure data

In [ ]:
Groups = pd.read_csv('../data/group_data.csv')

# Setting numerical values
group_dict =  {'admixed': 5,
               'asia': 2,
               'central_europe': 5,
               'germany': 4,
               'italy_balkan_caucasus': 5,
               'north_sweden': 1,
               'relict': 3,
               'south_sweden': 0,
               'spain': 5,
               'western_europe': 5}

Groups.loc[:,'group'] = [group_dict[i] for i in Groups.loc[:,'group']]

Groups = Groups[~Groups['ACCESSION'].isin([acc for acc in Groups.ACCESSION if acc not in STR_data.columns])]

Groups[['group', 'ACCESSION']].set_index('ACCESSION').T.to_csv('../data/matrixEQTL_covariate.csv', sep = '\t')

This accession is not in the group data set: SRR1945934

In [ ]:
for i in STR_data.columns:
    if i not in set(Groups.ACCESSION):
        print i

These frames were used for running 'matrixEQTL' in R

Loading results. This is **Supplementary Data 2**

In [ ]:
obs = pd.read_csv('../data/cis_norm_obs_group.csv')
con = pd.read_csv('../data/cis_obs_group_control.csv')

# Removing invalid tests
obs = obs[obs['statistic']!=0]
con = con[con['statistic']!=0]

## Figure 4a. Gene expression QQ plot

In [ ]:
fig, ax = subplots(figsize = (4,4))

control_sort = sort(con.pvalue)
real_sort = sort(obs.pvalue)

obs_y = [-log10(i) for i in real_sort]
obs_x = [-log10(i) for i in sorted(linspace(0, 1, len(real_sort)))]
control_y = [-log10(i) for i in control_sort]
control_x = [-log10(i) for i in sorted(linspace(0, 1, len(control_sort)))]


scatter(obs_x, obs_y, color = 'Red', s = 3)
scatter(control_x, control_y, color = 'Black', s = 3, alpha = 0.5)
plot([0,4.5],[0,4.5], color="gray")

xlim(0, 4)

ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_xticklabels(ax.get_xticks(), size=15)
ax.set_yticklabels(ax.get_yticks(), size=15)
ax.legend(fontsize = 16, loc = 'upper_left')
tight_layout()
savefig('../Figures/norm_cis_eQTL_QQ_plot.pdf', type = 'pdf', transparent = True)
savefig('../Figures/norm_cis_eQTL_QQ_plot.png', type = 'png', transparent = True, dpi = 300)


show()

Plotting expression STRs by genomic context

Making frame for plotting

In [ ]:
# Adding where
where = Repeats[Repeats['CHR_START'].isin(obs.snps)][['WHERE', 'CHR_START']].drop_duplicates()
obs.rename(columns = {'snps' : 'CHR_START'}, inplace = True)
obs_where = pd.merge(obs, where)

# Filtering for significant loci
sig_obs_where = obs_where[obs_where['FDR']< 0.05]

# For plotting direction
sig_obs_where.beta = [1 if x >= 0 else -1 for x in sig_obs_where.beta]

# Standard error
sig_obs_where['beta_se'] = sig_obs_where.beta / sig_obs_where.statistic

# Frame
data = sig_obs_where[['WHERE', 'beta', 'CHR_START']].pivot_table('beta', 'WHERE', 'CHR_START').replace(np.nan, 0)

Dividing by direction of effect

In [ ]:
CDS_neg = Counter(data.loc['CDS'])[-1.0]
CDS_pos = Counter(data.loc['CDS'])[1]
CDS_all = Counter(data.loc['CDS'])[0]
INTRON_neg = Counter(data.loc['INTRON'])[-1.0]
INTRON_pos = Counter(data.loc['INTRON'])[1]
INTRON_all = Counter(data.loc['INTRON'])[0]
UTR5_neg = Counter(data.loc['FIVE_PRIME_UTR'])[-1.0]
UTR5_pos = Counter(data.loc['FIVE_PRIME_UTR'])[1]
UTR5_all = Counter(data.loc['FIVE_PRIME_UTR'])[0]
UTR3_neg = Counter(data.loc['THREE_PRIME_UTR'])[-1.0]
UTR3_pos = Counter(data.loc['THREE_PRIME_UTR'])[1]
UTR3_all = Counter(data.loc['THREE_PRIME_UTR'])[0]
PROMOTER_neg = Counter(data.loc['PROMOTER'])[-1.0]
PROMOTER_pos = Counter(data.loc['PROMOTER'])[1]
PROMOTER_all = Counter(data.loc['PROMOTER'])[0]
SPANNING_neg = Counter(data.loc['SPANNING'])[-1.0]
SPANNING_pos = Counter(data.loc['SPANNING'])[1]
SPANNING_all = Counter(data.loc['SPANNING'])[0]

Dictionary to keep information

In [ ]:
frac_dict = {}
for feature in ['CDS', 'INTRON', 'FIVE_PRIME_UTR', 'THREE_PRIME_UTR', 'PROMOTER', 'SPANNING']:
    tested = float(len(set(Repeats[Repeats['WHERE']==feature].CHR_START)))
    sig = len(set(sig_obs_where[sig_obs_where['WHERE']==feature].CHR_START))
    frac = (sig / tested) * 100
    frac_dict.update({feature : frac})

In [ ]:
NEG = sig_obs_where[sig_obs_where['beta']==-1]
POS = sig_obs_where[sig_obs_where['beta']==1]

## Figure 4b. Direction of effect

In [ ]:
f, ax = subplots(frameon = True)
a = [CDS_neg, INTRON_neg, UTR5_neg, UTR3_neg, PROMOTER_neg, SPANNING_neg]
b = [CDS_pos, INTRON_pos, UTR5_pos, UTR3_pos, PROMOTER_pos, SPANNING_pos] 

ax.bar([0,0.5,1,1.5, 2.0, 2.5], a, width=0.125, color = '#d95f0e', label = 'Negative correlation', linewidth = 1, edgecolor = 'Black')
ax.bar([0.125,0.625,1.125,1.625, 2.125, 2.625], b, width = 0.125, color = '#fff7bc', label = 'Positive correlation', linewidth = 1, edgecolor = 'Black')
xticks([tick + 0.56 for tick in ax.get_xticks()])
ax.set_xticklabels(['CDS', 'Intron', '5\'UTR', '3\'UTR', 'Promoter', 'Spanning'])
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
#ylim(0, 65)
xlim(-0.3, 3)
#ax2 = ax.twinx()
#ax2.spines["top"].set_visible(False)
#ax2.yaxis.set_label_position("right")


ax.legend(loc = 'upper right')
savefig('../Figures/eQTL_results_norm_barplot.pdf', type = 'pdf', transparent = True)
savefig('../Figures/eQTL_results_norm_barplot.png', type = 'pdf', transparent = True, dpi = 300)


show()

In [ ]:
# Datasets
negs = [CDS_neg, INTRON_neg, UTR5_neg, UTR3_neg, PROMOTER_neg, SPANNING_neg]
poss = [CDS_pos, INTRON_pos, UTR5_pos, UTR3_pos, PROMOTER_pos, SPANNING_pos]
texts = ['CDS', 'INTRON', 'UTR5', 'UTR3', 'PROMOTER', 'SPANNING']

# Simulations
observed = []
expected = []
for i in range(len(negs)):
    neg = negs[i]
    pos = poss[i]
    observed.append(neg)
    exp = float(neg + pos) / 2
    expected.append(exp)

print scipy.stats.chisquare(observed, f_exp=expected)

No statistically significant bias in direction of effect

## Figure 4c. Percentage of eSTRs in genomic features

In [ ]:
f, ax = subplots(frameon = True, figsize = (4,4))
c = [frac_dict['CDS'], frac_dict['INTRON'], frac_dict['FIVE_PRIME_UTR'], frac_dict['THREE_PRIME_UTR'], frac_dict['PROMOTER'], frac_dict['SPANNING']]

ax.bar([0,1,2,3,4,5], c, width = 0.7, color = '#fec44f', label = 'eSTRs in group (%)', linewidth = 1, edgecolor = 'Black')
ax.set_xticklabels(['', 'CDS', 'Intron', '5\'UTR', '3\'UTR', 'Promoter', 'Spanning'], size = 15, rotation = 70)
ax.set_yticklabels(ax.get_yticks(), size=15)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
tight_layout()
savefig('../Figures/eQTL_results_norm_barplot_perc.pdf', type = 'pdf', transparent = True)
savefig('../Figures/eQTL_results_norm_barplot_perc.png', type = 'pdf', transparent = True, dpi = 300)
show()



## Figure 4d. Top 5 eSTRs

Gene expression data downloaded from https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE80744

In [ ]:
f, axes = subplots(nrows = 1, ncols = 5, figsize = (25,4), sharey = False)
counter = -1

boxprops = dict(linestyle='-', linewidth=3, color='darkorange')
flierprops = dict(marker='.', markerfacecolor='black', markersize=5,
                  linestyle='none')
medianprops = dict(linestyle='-', linewidth=3, color='black')

sig_res = obs[obs['FDR']<= 0.05]

for row in sig_res.sort_values(by = 'FDR', ascending = True).head(5).itertuples():
        counter += 1
        ax = axes.flatten()[counter]
        y = STR_data.reset_index()[STR_data.reset_index()['CHR_START']==row.CHR_START].dropna(axis = 1).iloc[0][1:].values
        columns = STR_data.reset_index()[STR_data.reset_index()['CHR_START']==row.CHR_START].dropna(axis = 1).columns[1:]

        filtered_Data = Data.drop([col for col in Data.columns[1:] if col not in columns], axis=1, inplace=False)
        x = filtered_Data[filtered_Data['GENE2']==mapper[row.CHR_START]].iloc[0].values[1:]
        variants = []
        expression = []
        df = pd.DataFrame([y, x]).T.dropna()

        for i in set(df[0]):
            variants.append(i)
            expression.append(list(df[df[0]==i][1].values))
        ax.boxplot(expression, positions = variants, boxprops = boxprops, flierprops=flierprops,
                   medianprops = medianprops)

        
        ticks = []
        for tick in ax.xaxis.get_ticklabels():
            ticks.append(tick.get_text())
        for tick in ax.xaxis.get_ticklabels():
            if tick.get_text() == min(ticks) or tick.get_text() == max(ticks):
                tick.set_fontsize(20)
            else:
                tick.set_visible(False)
        ax.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
        setp(ax.yaxis.get_ticklabels(), size = 20)


        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)
tight_layout()
savefig('../Figures/Top5_eSTRs_by_FDR.pdf', type = 'pdf', transparent = True)
savefig('../Figures/Top5_eSTRs_by_FDR.png', type = 'png', transparent = True)
show()